# Dog Breed Identification

In [20]:
#Importing required libraries.
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from IPython.display import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import imread

In [5]:
#Check GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
#Importing dog breed datasets's labels
labels_csv = pd.read_csv("DogDataset/labels.csv")
print (labels_csv.describe())             #labels descriptions   
print (labels_csv.head())                 #Top 5 rows of labels csv


                                      id               breed
count                              10222               10222
unique                             10222                 120
top     0857307dddb1b41288887ef2338af6df  scottish_deerhound
freq                                   1                 126
                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [9]:
#Images in each breed
labels_csv["breed"].value_counts().median()

82.0

In [11]:
#Creating list of file names from the labels's csv file
filename = []
for fname in labels_csv["id"]:
  path = "DogDataset/train/" + fname + ".jpg"
  filename.append(path)

In [12]:
#Verify the all the labels in csv file
if len(os.listdir("DogDataset/train/")) == len(filename):
  print ("Verified all files.")
else:
  print ("Missing some files!!")

Verified all files.


In [13]:
labels = labels_csv["breed"].to_numpy()
labels

array(['boston_bull', 'dingo', 'pekinese', ..., 'airedale',
       'miniature_pinscher', 'chesapeake_bay_retriever'], dtype=object)

In [14]:
if len(labels) == len(filename):
  print ("All labels verified")
else:
  print ("Missing some files")

All labels verified


In [15]:
#uniques labels
unique_breeds = np.unique(labels)
boolean_labels = [label == unique_breeds for label in labels]

In [17]:
# Creating validation set
x = filename
y = boolean_labels

In [18]:
x_train , x_val, y_train, y_val = train_test_split(x[:1000],
                                                   y[:1000],
                                                   test_size = 0.2,
                                                   random_state=42)
len(x_train), len(x_val), len(y_train), len(y_val)

(800, 200, 800, 200)

# Preprocessing Images 
## take an image filepath
## use tensflow to read file and save it to a image
## turn our image to tensor
## normalize our image 
## resize the image to be a shape of (224,224)
## return the modified image

In [21]:
IMG_SIZE = 224

def pre_process_image(image_path):
    #read in a image file
    image = tf.io.read_file(image_path)
    #turn image to numerical tensor with 3 color RGB
    image = tf.image.decode_jpeg(iamge, channels=3)
    #convert the color channel value from 0-255 to 0-1 value
    image = tf.image.convert_image_dtype(image, tf.float32)
    #resize our image to our desired value (244,244)
    image = tf.image.resize(image, size = [IMAGE_SIZE, IMAGE_SIZE])
    #return the image
    return image

In [22]:
#turning our data into batches
def get_image_label(image_path):
    image = pre_process_image(image_path)
    
    return image, label

In [23]:
BATCH_SIZE = 32

def create_data_batches(X, y = None, batch_size = BATCH_SIZE, valid_data = False, test_data = False):

  # If the data is the test data set, we don't have labels
  if test_data:
    print('Creating test data batches . . . .')
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))   # Only filepaths no labels
    data_batch = data.map(pre_process_image).batch(BATCH_SIZE)
    return data_batch

    # If the data is a valid data set, we don't need to shuffle it

  elif valid_data:
    print("Creating validation data batches . . . ")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    data_batch = data.map(pre_process_image).batch(BATCH_SIZE) #Also preprocesses the images 
    return data_batch

  else:
    print("Creating training data batches . . . . ")    
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    # Shuffling the pathnames before mapping and processing 
    data = data.shuffle(buffer_size = len(X))

    data = data.map(get_image_label)

    data_batch = data.batch(BATCH_SIZE)
    return data_batch

In [25]:
train_data = create_data_batches(x_train, y_train)
valid_data = create_data_batches(x_val, y_val, valid_data = True)

Creating training data batches . . . . 


TypeError: in converted code:


    TypeError: tf__get_image_label() takes 1 positional argument but 2 were given
